# Continuous Control

---

Congratulations for completing the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control an agent in a more challenging environment, where the goal is to train a creature with four arms to walk forward.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Crawler.app"`
- **Windows** (x86): `"path/to/Crawler_Windows_x86/Crawler.exe"`
- **Windows** (x86_64): `"path/to/Crawler_Windows_x86_64/Crawler.exe"`
- **Linux** (x86): `"path/to/Crawler_Linux/Crawler.x86"`
- **Linux** (x86_64): `"path/to/Crawler_Linux/Crawler.x86_64"`
- **Linux** (x86, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86"`
- **Linux** (x86_64, headless): `"path/to/Crawler_Linux_NoVis/Crawler.x86_64"`

For instance, if you are using a Mac, then you downloaded `Crawler.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Crawler.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
env_info = env.reset(train_mode=True)[brain_name]

In [6]:
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from agent import Agent


In [7]:
load_agent = False

def load_saved_agent(local_actor_pth, local_critic_pth, target_actor_pth, target_critic_pth):
    local_actor = torch.load_agent(local_actor_pth)
    local_critic = torch.load_agent(local_critic_pth)
    target_actor = torch.load_agent(target_actor_pth)
    target_critic = torch.load_agent(target_critic_pth)
    
    loaded_agent = Agent(action_size, state_size, False, num_agents, random_seed=0)
    
    loaded_agent.local_actor.load_state_dict(local_actor)
    loaded_agent.local_critic.load_state_dict(local_critic)
    loaded_agent.target_actor.load_state_dict(target_actor)
    loaded_agent.target_critic.load_state_dict(target_critic)
    
    return loaded_agent

if load_agent:
    agent = load_saved_agent('checkpoint_local_actor.pth', 'checkpoint_local_critic.pth', 
                       'checkpoint_target_actor.pth', 'checkpoint_target_critic.pth')
    print('Agent loaded')
else:
    agent = Agent(action_size, state_size, True, num_agents, random_seed=0)
    print('Agent created')

Agent created


In [8]:
def d4pg(n_episodes=1000, print_every=100):
    scores = []
    scores_deque = deque(maxlen=print_every)
    roll_avg_score = []

    max_score = 40
    current_score = 0

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations
        agent.reset()
        score = np.zeros(num_agents)
        while True:
            action = agent.act(state, current_score, max_score)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards
            done = env_info.local_done

            score += reward
            agent.step(state, action, reward, next_state, done)
            state = next_state
            if np.any(done):  # check to see if any episodes are done
                break

        score = np.mean(score)
        scores_deque.append(score)
        scores.append(score)
        roll_avg_score.append(np.mean(scores_deque))

        current_score = np.mean(scores_deque)

        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        
        if i_episode % 100 == 0:                        
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
        if i_episode % 10 == 0:            
            torch.save(agent.local_actor.state_dict(), 'checkpoint_local_actor.pth')           # save local actor
            torch.save(agent.target_actor.state_dict(), 'checkpoint_target_actor.pth')         # save target actor 
            torch.save(agent.local_critic.state_dict(), 'checkpoint_local_critic.pth')         # save local critic
            torch.save(agent.target_critic.state_dict(), 'checkpoint_target_critic.pth')       # target critic
                        
        if 30 < current_score and 99 < len(scores_deque):
            print('Target average reward achieved!')
            torch.save(agent.local_actor.state_dict(), 'checkpoint_local_actor.pth')           # save local actor
            torch.save(agent.target_actor.state_dict(), 'checkpoint_target_actor.pth')         # save target actor 
            torch.save(agent.local_critic.state_dict(), 'checkpoint_local_critic.pth')         # save local critic
            torch.save(agent.target_critic.state_dict(), 'checkpoint_target_critic.pth')       # target critic
            break


    return scores, roll_avg_score

scores, roll_avg_scores = d4pg()
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.plot(np.arange(1, len(roll_avg_scores)+1), roll_avg_scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 26	Average Score: 0.01	Score: 0.01

KeyboardInterrupt: 